In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding= 'unicode_escape')

In [4]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [5]:
df.emotion_in_tweet_is_directed_at.value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [6]:
df.emotion_in_tweet_is_directed_at.isna().sum()

5802

In [7]:
df.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

In [18]:
df.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [17]:
df[df.emotion_in_tweet_is_directed_at.isna()]['is_there_an_emotion_directed_at_a_brand_or_product'].value_countscounts()

No emotion toward brand or product    5298
Positive emotion                       306
I can't tell                           147
Negative emotion                        51
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [9]:
df[df.tweet_text.isna() == True]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
6,NaN,NaN,No emotion toward brand or product


In [11]:
df.dropna()

tweet_text                                                                           NaN
emotion_in_tweet_is_directed_at                                                      NaN
is_there_an_emotion_directed_at_a_brand_or_product    No emotion toward brand or product
Name: 6, dtype: object

In [16]:
df.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [23]:
df1 = df[(df.is_there_an_emotion_directed_at_a_brand_or_product == 'Positive emotion') | 
         (df.is_there_an_emotion_directed_at_a_brand_or_product == 'Negative emotion') ]

In [25]:
df1.isna().sum()

tweet_text                                              0
emotion_in_tweet_is_directed_at                       357
is_there_an_emotion_directed_at_a_brand_or_product      0
dtype: int64

In [27]:
# rename columns  
df1.rename(columns={'tweet_text': 'text', 'emotion_in_tweet_is_directed_at': 'brand', 
                   'is_there_an_emotion_directed_at_a_brand_or_product': 'sentiment'}, inplace=True)


# condence branch values to a binary category
brand_replace = {'iPad': 'apple', 'Apple': 'apple', 'iPad or iPhone App': 'apple', 'Google': 'google', 'iPhone': 'apple',
       'Other Google product or service': 'google', 'Android App': 'google', 'Android': 'google',
       'Other Apple product or service': 'apple'}
df1.brand.replace(brand_replace, inplace=True)

# clean sentiment value names
sentiment_replace = {'No emotion toward brand or product': 'neutral', 'Positive emotion': 'positive', 
                     'Negative emotion': 'negative'}
df1.sentiment.replace(sentiment_replace, inplace=True)

In [28]:
df1

,text,brand,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,apple,negative
1,@jessedee Know about @fludapp ? Awesome iPad/i...,apple,positive
2,@swonderlin Can not wait for #iPad 2 also. The...,apple,positive
3,@sxsw I hope this year's festival isn't as cra...,apple,negative
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,google,positive
...,...,...,...
9077,@mention your PR guy just convinced me to swit...,apple,positive
9079,&quot;papyrus...sort of like the ipad&quot; - ...,apple,positive
9080,Diller says Google TV &quot;might be run over ...,google,negative
9085,I've always used Camera+ for my iPhone b/c it ...,apple,positive


In [30]:
df1.brand.value_counts()

apple     2337
google     854
Name: brand, dtype: int64

In [31]:
df1.isna().sum()

text           0
brand        357
sentiment      0
dtype: int64